Example notebook on how to use the Ensemble and Analysis objects

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Load the data

In [3]:
# to be changed, depeding on how the data is stored
mask = xr.open_dataarray('../../data_events/evaluation_mask.nc')
nowcasts = xr.open_dataarray('../../STEPS/data/predictions_convective.zarr',
                                chunks = {'central_time': 1, 'lead_time': 1, 'member': -1, 'y': -1, 'x': -1}).where(mask, drop = True).isel(central_time = 0)
observations = xr.open_dataarray('../../data_events/convective_events.nc',
                                 chunks = {'central_time': 1, 'lead_time': 1, 'y': -1, 'x': -1}).isel(lead_time = np.arange(4, 24)).where(mask, drop = True).isel(central_time = 0)

The nowcasts and the observations are supposed to have the following shape

In [4]:
nowcasts.dims

('lead_time', 'member', 'y', 'x')

In [5]:
observations.dims

('lead_time', 'y', 'x')

# Create an Ensemble object

The Ensemble objects are the main objects in dynnow. They are created by passing a xr.Dataset object:

In [17]:
from dynnow.ensemble import Ensemble
ds = xr.Dataset(data_vars = {'nowcasts': nowcasts, 'observation': observations})
original = Ensemble(ds)

All computations are done lazily by dask, until .compute() is called:

In [7]:
original

<xarray.Ensemble> Size: 870MB
Dimensions:        (lead_time: 20, member: 50, x: 320, y: 320)
Coordinates:
  * lead_time      (lead_time) timedelta64[s] 160B 00:05:00 ... 01:40:00
  * member         (member) int64 400B 1 2 3 4 5 6 7 8 ... 44 45 46 47 48 49 50
  * x              (x) float64 3kB 4.905e+05 4.915e+05 ... 8.085e+05 8.095e+05
  * y              (y) float64 3kB 8.095e+05 8.085e+05 ... 4.915e+05 4.905e+05
    central_time   datetime64[ns] 8B 2017-04-26T13:00:00
    lat            (y, x) float64 819kB 52.07 52.07 52.07 ... 49.21 49.21 49.21
    lon            (y, x) float64 819kB 2.043 2.058 2.072 ... 6.53 6.543 6.557
    time           (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    nowcasts       (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
    observation    (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    ensemble_mean  (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    error          (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
Attributes:
    ensemble:       True
    spatial_shape:  (320, 320)

# Compute scores of the ensemble

The spectral analysis carried out in the paper can be automatically added to the Ensemble object with

In [13]:
original.spectral_analyze()
original

<xarray.Ensemble> Size: 870MB
Dimensions:                 (lead_time: 20, member: 50, x: 320, y: 320,
                             scale: 160)
Coordinates:
  * lead_time               (lead_time) timedelta64[s] 160B 00:05:00 ... 01:4...
  * member                  (member) int64 400B 1 2 3 4 5 6 ... 46 47 48 49 50
  * x                       (x) float64 3kB 4.905e+05 4.915e+05 ... 8.095e+05
  * y                       (y) float64 3kB 8.095e+05 8.085e+05 ... 4.905e+05
  * scale                   (scale) float64 1kB inf 320.0 160.0 ... 2.025 2.013
    central_time            datetime64[ns] 8B 2017-04-26T13:00:00
    lat                     (y, x) float64 819kB 52.07 52.07 ... 49.21 49.21
    lon                     (y, x) float64 819kB 2.043 2.058 ... 6.543 6.557
    time                    (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    nowcasts                (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
    observation             (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    ensemble_mean           (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    error                   (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    spectral_variance       (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
    spectral_error          (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
    spectral_ensemble_mean  (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
    spectral_observation    (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
Attributes:
    ensemble:       True
    spatial_shape:  (320, 320)

Other scores can also be added with

In [14]:
original.ensemble_scores()
original

<xarray.Ensemble> Size: 2GB
Dimensions:                     (lead_time: 20, member: 50, x: 320, y: 320,
                                 scale: 160, order: 49, bins_center: 100,
                                 threshold_FSS: 7, scale_FSS: 9)
Coordinates:
  * lead_time                   (lead_time) timedelta64[s] 160B 00:05:00 ... ...
  * member                      (member) int64 400B 1 2 3 4 5 ... 46 47 48 49 50
  * x                           (x) float64 3kB 4.905e+05 ... 8.095e+05
  * y                           (y) float64 3kB 8.095e+05 ... 4.905e+05
  * scale                       (scale) float64 1kB inf 320.0 ... 2.025 2.013
  * bins_center                 (bins_center) float64 800B -0.99 -0.97 ... 0.99
  * threshold_FSS               (threshold_FSS) float64 56B -6.021 ... 12.04
  * scale_FSS                   (scale_FSS) int64 72B 2 3 5 9 17 33 65 129 257
    central_time                datetime64[ns] 8B 2017-04-26T13:00:00
    lat                         (y, x) float64 819kB 52.07 52.07 ... 49.21 49.21
    lon                         (y, x) float64 819kB 2.043 2.058 ... 6.543 6.557
    time                        (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: order
Data variables: (12/15)
    nowcasts                    (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
    observation                 (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    ensemble_mean               (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    error                       (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    spectral_variance           (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
    spectral_error              (lead_time, scale) float64 26kB dask.array<chunksize=(1, 160), meta=np.ndarray>
    ...                          ...
    eigenvec                    (lead_time, order, y, x) float64 803MB dask.array<chunksize=(1, 49, 320, 320), meta=np.ndarray>
    spectral_eigenvec           (lead_time, order, scale) float64 1MB dask.array<chunksize=(1, 49, 160), meta=np.ndarray>
    cosine_error_members        (lead_time, order) float64 8kB dask.array<chunksize=(1, 49), meta=np.ndarray>
    cosine_error_projection     (lead_time) float64 160B dask.array<chunksize=(1,), meta=np.ndarray>
    histos_projection_eigenvec  (lead_time, order, bins_center) float64 784kB dask.array<chunksize=(1, 49, 100), meta=np.ndarray>
    FSS                         (lead_time, threshold_FSS, scale_FSS) float64 10kB dask.array<chunksize=(1, 7, 9), meta=np.ndarray>
Attributes:
    ensemble:       True
    spatial_shape:  (320, 320)

# Creation of surrogate ensembles and Analysis objects

To generate surrogate nowcasts:

In [11]:
maaft = original.generate_surrogates('maaft') # 'maaft', 'hist' and 'spectral' are supported here
maaft

<xarray.Ensemble> Size: 870MB
Dimensions:        (lead_time: 20, member: 50, y: 320, x: 320)
Coordinates:
  * lead_time      (lead_time) timedelta64[s] 160B 00:05:00 ... 01:40:00
  * member         (member) int64 400B 1 2 3 4 5 6 7 8 ... 44 45 46 47 48 49 50
  * y              (y) float64 3kB 8.095e+05 8.085e+05 ... 4.915e+05 4.905e+05
  * x              (x) float64 3kB 4.905e+05 4.915e+05 ... 8.085e+05 8.095e+05
    central_time   datetime64[ns] 8B 2017-04-26T13:00:00
    lat            (y, x) float64 819kB 52.07 52.07 52.07 ... 49.21 49.21 49.21
    lon            (y, x) float64 819kB 2.043 2.058 2.072 ... 6.53 6.543 6.557
    time           (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    nowcasts       (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
    observation    (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    ensemble_mean  (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    error          (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
Attributes:
    ensemble:       True
    ensemble_type:  maaft
    spatial_shape:  (320, 320)

In [12]:
# maaft is an Ensemble object as well, so the scores can be computed with
maaft.ensemble_scores()
maaft

<xarray.Ensemble> Size: 2GB
Dimensions:                     (lead_time: 20, member: 50, y: 320, x: 320,
                                 order: 49, scale: 160, bins_center: 100,
                                 threshold_FSS: 7, scale_FSS: 9)
Coordinates:
  * lead_time                   (lead_time) timedelta64[s] 160B 00:05:00 ... ...
  * member                      (member) int64 400B 1 2 3 4 5 ... 46 47 48 49 50
  * y                           (y) float64 3kB 8.095e+05 ... 4.905e+05
  * x                           (x) float64 3kB 4.905e+05 ... 8.095e+05
  * scale                       (scale) float64 1kB inf 320.0 ... 2.025 2.013
  * bins_center                 (bins_center) float64 800B -0.99 -0.97 ... 0.99
  * threshold_FSS               (threshold_FSS) float64 56B -6.021 ... 12.04
  * scale_FSS                   (scale_FSS) int64 72B 2 3 5 9 17 33 65 129 257
    central_time                datetime64[ns] 8B 2017-04-26T13:00:00
    lat                         (y, x) float64 819kB 52.07 52.07 ... 49.21 49.21
    lon                         (y, x) float64 819kB 2.043 2.058 ... 6.543 6.557
    time                        (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: order
Data variables:
    nowcasts                    (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
    observation                 (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    ensemble_mean               (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    error                       (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
    eigenval                    (lead_time, order) float64 8kB dask.array<chunksize=(1, 49), meta=np.ndarray>
    eigenvec                    (lead_time, order, y, x) float64 803MB dask.array<chunksize=(1, 49, 320, 320), meta=np.ndarray>
    spectral_eigenvec           (lead_time, order, scale) float64 1MB dask.array<chunksize=(1, 49, 160), meta=np.ndarray>
    cosine_error_members        (lead_time, order) float64 8kB dask.array<chunksize=(1, 49), meta=np.ndarray>
    cosine_error_projection     (lead_time) float64 160B dask.array<chunksize=(1,), meta=np.ndarray>
    histos_projection_eigenvec  (lead_time, order, bins_center) float64 784kB dask.array<chunksize=(1, 49, 100), meta=np.ndarray>
    FSS                         (lead_time, threshold_FSS, scale_FSS) float64 10kB dask.array<chunksize=(1, 7, 9), meta=np.ndarray>
Attributes:
    ensemble:       True
    ensemble_type:  maaft
    spatial_shape:  (320, 320)

Now, we have two objects (original and maaft). Calling .compute() on each of them separately will trigger two distinct dask computations. To make dask handle everything at once, the trick is to embed these Ensemble objects in a xr.DataTree object. A special subclass has been designed for this:

In [20]:
from dynnow.analysis import Analysis
a = Analysis(xr.Dataset(original.coords), children = {'original': xr.DataTree(original), 'maaft': xr.DataTree(maaft)})
a

<xarray.DataTree>
Group: /
│   Dimensions:       (lead_time: 20, member: 50, x: 320, y: 320)
│   Coordinates:
│     * lead_time     (lead_time) timedelta64[s] 160B 00:05:00 00:10:00 ... 01:40:00
│     * member        (member) int64 400B 1 2 3 4 5 6 7 8 ... 44 45 46 47 48 49 50
│     * x             (x) float64 3kB 4.905e+05 4.915e+05 ... 8.085e+05 8.095e+05
│     * y             (y) float64 3kB 8.095e+05 8.085e+05 ... 4.915e+05 4.905e+05
│       central_time  datetime64[ns] 8B 2017-04-26T13:00:00
│       lat           (y, x) float64 819kB 52.07 52.07 52.07 ... 49.21 49.21 49.21
│       lon           (y, x) float64 819kB 2.043 2.058 2.072 ... 6.53 6.543 6.557
│       time          (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
├── Group: /original
│       Dimensions:        (lead_time: 20, member: 50, y: 320, x: 320)
│       Coordinates:
│           central_time   datetime64[ns] 8B 2017-04-26T13:00:00
│           lat            (y, x) float64 819kB 52.07 52.07 52.07 ... 49.21 49.21 49.21
│           lon            (y, x) float64 819kB 2.043 2.058 2.072 ... 6.53 6.543 6.557
│           time           (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
│       Data variables:
│           nowcasts       (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
│           observation    (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
│           ensemble_mean  (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
│           error          (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
│       Attributes:
│           ensemble:       True
│           spatial_shape:  (320, 320)
└── Group: /maaft
        Dimensions:                     (lead_time: 20, member: 50, y: 320, x: 320,
                                         order: 49, scale: 160, bins_center: 100,
                                         threshold_FSS: 7, scale_FSS: 9)
        Coordinates:
          * scale                       (scale) float64 1kB inf 320.0 ... 2.025 2.013
          * bins_center                 (bins_center) float64 800B -0.99 -0.97 ... 0.99
          * threshold_FSS               (threshold_FSS) float64 56B -6.021 ... 12.04
          * scale_FSS                   (scale_FSS) int64 72B 2 3 5 9 17 33 65 129 257
            central_time                datetime64[ns] 8B 2017-04-26T13:00:00
            lat                         (y, x) float64 819kB 52.07 52.07 ... 49.21 49.21
            lon                         (y, x) float64 819kB 2.043 2.058 ... 6.543 6.557
            time                        (lead_time) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
        Dimensions without coordinates: order
        Data variables:
            nowcasts                    (lead_time, member, y, x) float64 819MB dask.array<chunksize=(1, 50, 320, 320), meta=np.ndarray>
            observation                 (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
            ensemble_mean               (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
            error                       (lead_time, y, x) float64 16MB dask.array<chunksize=(1, 320, 320), meta=np.ndarray>
            eigenval                    (lead_time, order) float64 8kB dask.array<chunksize=(1, 49), meta=np.ndarray>
            eigenvec                    (lead_time, order, y, x) float64 803MB dask.array<chunksize=(1, 49, 320, 320), meta=np.ndarray>
            spectral_eigenvec           (lead_time, order, scale) float64 1MB dask.array<chunksize=(1, 49, 160), meta=np.ndarray>
            cosine_error_members        (lead_time, order) float64 8kB dask.array<chunksize=(1, 49), meta=np.ndarray>
            cosine_error_projection     (lead_time) float64 160B dask.array<chunksize=(1,), meta=np.ndarray>
            histos_p